In [2]:
import gurobipy as gp

In [3]:
M = 5
N = 10

cs_i = [ 400 , 100 , 600 , 1000 , 150 ]
ct_jk = [ 2 , 10 , 50 , 3 , 5 , 6 , 15 , 30 , 18 , 30 ]

P_j = [ 100 , 30 , 5 , 130 , 55 , 40 , 15 , 30 , 50 , 10 ]
c_jk = [ 10 , 50 , 500 , 15 , 25 , 30 , 75 , 150 , 90 , 80 ]


es_i = [ 1 , 1 , 1 , 1 , 1 ]
et_jk = [  1 , 1 , 1 , 1 , 1 , 1 , 1 , 1 , 1 , 1 ]

In [7]:
# Rotulos dos clientes
clients = []
for j in range(N):
    clients.append(f"Client_{j}")

# Rotulos das squads
squads = []
for i in range(M):
    squads.append(f"Squad_{i}")

# Rotulos das tarefas
tasks = []
for j in range(N):
    tasks_j  = []
    for k in range(P_j[j]):
        tasks_j.append(f"Task_{j}_{k}")
    
    tasks.append(tasks_j)


In [29]:
# Dicionário das capacidades das Squads
capacities = {}
specialty = {}
for i, cap in enumerate(cs_i):
    label = squads[i]
    capacities[label] = cap
    specialty[label] = es_i[i]

# Dicionário da quantidade de tarefas por cliente
qtd_tasks = {}
for j, qtd in enumerate(P_j):
    label = clients[j]
    qtd_tasks[label] = qtd

# Dicionário de valor, Story Points e especialidade de cada tarefa
values = []
story_points = []
req_expertness = []
for j, qtd in enumerate(P_j):
    value = {}
    story_point = {}
    req_expertise = {}
    for k, label in enumerate(tasks[j]):
        value[label] = c_jk[j]
        story_point[label] = ct_jk[j]
        req_expertise[label] = et_jk[j]
    
    values.append(value)
    story_points.append(story_point)
    req_expertness.append(req_expertise)

In [22]:
# Criando modelo
model = gp.Model()

In [23]:
# Inserindo variáveis de decisão
u_ijk = []
for j in range(N):
    u_ij = model.addVars(squads, tasks[j], vtype=gp.GRB.BINARY) # Ambos os vetores tem apenas rotulos e não valores
    u_ijk.append(u_ij)

In [25]:
# Define a função objetivo 1 do lucro
j = 0
model.setObjective(
    gp.quicksum(u_ijk[j][i,k] * values[j][k] for j in range(N) for k in tasks[j] for i in squads ),
    sense=gp.GRB.MAXIMIZE
)

# Definindo as restrições

In [36]:
## Garantindo que nenhuma squad tenha sua capacidade máxima exedida
c1 = model.addConstrs(
    gp.quicksum(u_ijk[j][i,k] * story_points[j][k] for j in range(N) for k in tasks[j]) <= capacities[i] for i in squads
)

In [37]:
## Garantindo que a especialidade da squad e tarefa sejam a mesma
c2 = model.addConstr(
    gp.quicksum(u_ijk[j][i,k] * (specialty[i] - req_expertness[j][k]) for j in range(N) for k in tasks[j] for i in squads ) == 0
)

In [43]:
## Garantindo que pelo menos uma tarefa por cliente seja atendida
c3 = model.addConstrs(
    gp.quicksum(u_ijk[j][i,k] for k in tasks[j] for i in squads) >= 1 for j in range(N) 
)

In [44]:
## Garantindo que apenas uma squad pegue a mesma tarefa
c4 = model.addConstrs(
    gp.quicksum(u_ijk[j][i,k] for i in squads) <= 1 for j in range(N) for k in tasks[j] 
)